<a href="https://colab.research.google.com/github/cathepsin/alpha_beta_ProtBERT/blob/main/protbert_burn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MM-BERT MLM Predictions With ProtBERT

In [34]:
#@title Settings upload

from google.colab import drive
from google.colab import files
import json

Settings_source = "From Computer" #@param {values: ["From Computer", "From Drive or Colab"]}
#@markdown If using a settings file from Drive or Colab, please specify the filepath below
Settings_filepath_in_drive_or_colab = "" #@param {type: "string"}
#@markdown Prompt to redownload a new settings file every time this program is run. Otherwise if a settings file is detected then it will be reused.
Redownload_settings = False #@param {type : "boolean"}

if Redownload_settings == False and 'all_settings' in globals():
  pass
elif Settings_source == "From Computer":
  filepath = next(iter(files.upload().keys()))
else:
  filepath = Settings_filepath_in_drive_or_colab

with open(filepath, "r") as fp:
  all_settings = json.loads(fp.read())
  configuration = all_settings["Configuration_Settings"]
  finetune = all_settings["Fine_Tune_Settings"]
  mlm = all_settings["MLM_Settings"]





In [35]:
#@title Setup, Imports, and Connect to Runtime
!pip install -q transformers
if  "completed_setup" not in globals():
  from termcolor import colored
  from transformers import BertTokenizer, BertForMaskedLM, pipeline
  from transformers import AdamW
  import os
  import torch
  from tqdm import tqdm
  from time import gmtime, strftime
  import re
  import copy as cp
  import sys
  import random
  import matplotlib.pyplot as plt
  drive.mount("/content/gdrive")
  tokenizer = BertTokenizer.from_pretrained('Rostlab/prot_bert')
  model = BertForMaskedLM.from_pretrained('Rostlab/prot_bert')
  completed_setup = True

use_custom_weights = configuration['use_custom_weights']
path_to_file_in_colab = configuration['path_to_file_in_colab']
fine_tune_model = configuration['fine_tune_model']
MLM_model = configuration['MLM_prediction']

if use_custom_weights == 'From Google Drive or Colab':
  #Make sure paths exist and are valid
  assert path_to_file_in_colab, 'No path given'
  assert os.path.isfile(path_to_file_in_colab), 'Invalid filepath'
  model.load_state_dict(torch.load(path_to_file_in_colab))
elif use_custom_weights == 'From computer':
  #prompt user to upload weight file
  model.load_state_dict(torch.load(files.upload()))


In [36]:
#@title Fine-tune model
fsettings = finetune['FT_Settings']
ssettings = finetune['Save_Settings']

fine_file = fsettings['fine_tune_file_source']
path_to_ft_file = fsettings['path_to_fine_tune_file']
percent_mask = fsettings['percent_to_mask']
max_len = fsettings['max_length']
batch_size = fsettings['max_length']
number_of_epochs = fsettings['number_of_epochs']
learning_rate = fsettings['learning_rate']

save_every = ssettings['save_every_number_of_steps']
save_to_drive = ssettings['save_final_to_drive']
download_to_computer = ssettings['download_final_state']

if save_to_drive:
  from google.colab import drive
  drive.mount('/content/gdrive')

def RunningAvg(lst):
  ret = []
  for i in range(len(lst)):
    try:
      ret.append(sum(lst[0:i])/len(lst[0:i]))
    except ZeroDivisionError:
      ret.append(0)
  return ret

def GetAccuracy(input_id, labels, logits, mask_label = 4):
  num_correct = 0
  total = 0
  for i in range(len(input_id)):
    for v in input_id[i]:
      if input_id[i][v] == mask_label:
        total += 1
        val = torch.argmax(torch.softmax(logits[i][v], -1))
        if labels[i][v] == val:
          num_correct += 1
  return num_correct / total

class ProtDataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings

  def __getitem__(self, index):
    return {key: val[index].clone().detach() for key, val in self.encodings.items()}

  def __len__(self):
    return len(self.encodings.input_ids)

if fine_tune_model:
  if fine_file == 'From Google Drive or Colab':
    fp = path_to_ft_file
  else:
    up = files.upload()
    fp = list(up.keys())[0]
    up.clear()

  with open(fp, 'r') as f:
    text = f.read().split('\n')
    for i in range(len(text)):
      text[i] = " ".join(text[i])

  print("Tokenizing file...")
  inputs = tokenizer(text, return_tensors='pt', max_length=max_len, truncation=True, padding='max_length')
  inputs['labels'] = inputs.input_ids.detach().clone()
  rand = torch.rand(inputs.input_ids.shape)
  mask_ar = (rand < percent_mask) * (inputs.input_ids != 2) * (inputs.input_ids != 0) * (inputs.input_ids != 3)

  selection = []
  for i in range(mask_ar.shape[0]):
    selection.append(torch.flatten(mask_ar[i].nonzero()).tolist())

  for i in range(mask_ar.shape[0]):
    inputs.input_ids[i, selection[i]] = 4

  dataset = ProtDataset(inputs)
  dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
  device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  if not torch.cuda.is_available():
    print("No CUDA enabled GPU available. Running on CPU")

  model.to(device);
  model.train();

  optim = torch.optim.AdamW(model.parameters(), lr=learning_rate)

  cnt = 1
  graph_count = []
  graph_loss = []
  graph_accuracies = []
  # nan = open('nan.txt', 'w')
  # debug = open('debug.txt', 'w')
  for epoch in range(number_of_epochs):
    loop = tqdm(dataloader, position=0, leave=True)
    for batch in loop:
      optim.zero_grad()
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)
      outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

      loss = outputs.loss
      loss.backward()
      optim.step()

      graph_count.append(cnt)
      graph_loss.append(loss.item())

      graph_accuracies.append(GetAccuracy(input_ids, labels, outputs['logits']))
      # debug.write(f"{cnt} {graph_accuracies[-1]} {loss}\n")
      # debug.write(f"\t{outputs['logits']}\n")
      # for l in outputs['logits']:
      #   for v in l:
      #     if True in torch.isnan(v):
      #       print(f"Nan value in {cnt}")

      # if cnt == 10:
      #   debug.close()
      #   raise
      # if cnt % save_every == 0:
      #   torch.save(model.state_dict(), '/content/save_state')
      #   if save_recent_state_to_drive:
      #     torch.save(model.state_dict(), path_to_most_recent + "/most_recent")

      #####################
           ####BETA####
      # if cnt % smart_steps == 0 and smart_learn and cnt != 0:
      #   avg = sum(graph_loss[-smart_steps:]) / smart_steps
      #   for l in optim.param_groups:
      #     l['lr'] /= (1 + avg)
      #####################
      cnt += 1
      loop.set_description(f'Epoch: {epoch}, Step: {cnt}, Accuracy: {graph_accuracies[-1]:.3f}, Loss: {loss.item():.8f}')
      loop.update(1)

  # debug.close()
  plt.plot(graph_count, graph_loss)
  plt.plot(graph_count, RunningAvg(graph_loss))
  plt.grid()
  plt.title("Fine-Tuning Loss")
  plt.xlabel("Step")
  plt.ylabel("Loss")

  plt.figure()
  plt.plot(graph_count, graph_accuracies)
  plt.grid()
  plt.title("Fine-Tuning Accuracy")
  plt.xlabel("Step")
  plt.ylabel("Accuracy")


  plt.figure()
  plt.title("Combined Graph")
  plt.xlabel("Step")
  plt.ylabel("Loss")
  plt.grid()
  plt2 = plt.twinx()
  plt.plot(graph_count, graph_loss)
  plt2.plot(graph_count, graph_accuracies)
  plt2.set_ylabel("Accuracy")
  plt2.set_ylim([0.0, 1.0])

  plt.legend(["Loss", "Accuracy"])
  torch.save(model.state_dict(), '/content/save_state_final')
  if save_to_drive:
    print("Saving to google drive is currently disabled. Please manually move the save state called 'save_state-final' to your drive")
    # from google.colab import drive
    # os.system(r'mv /content/save ' + colab_save_path)
  if download_to_computer:
    torch.save(model.state_dict(), '/content/save_state')
    files.download('/content/save_state')

In [ ]:
#@title Predictions
###
seqsettings = mlm['Sequence_Settings']
asettings = mlm['Acceptance_Settings']
bsettings = mlm['Burn-in_Settings']
rsettings = mlm['Results_and_Display_Settings']
lsettings = mlm['Logging']

write_log = lsettings['write_log_file']
num_to_log = lsettings['number_to_log']

order = rsettings['order']
get_score_average = rsettings['get_score_average']
get_score_entropy = rsettings['get_score_entropy']
spaces_in_output = rsettings['spaces_in_output']
display_iterations = rsettings['display_iterations']
display_num_different = rsettings['display_num_different']
display_score_average = rsettings['display_score_average']
display_score_entropy = rsettings['display_score_entropy']

use_random_sequence = seqsettings['use_random_sequence']
random_sequence_length = seqsettings['random_sequence_length']
original = seqsettings['original_sequence']
masked = seqsettings['masked_sequence  ']
match_original = seqsettings['mask_all_residues']
process_batch = seqsettings['process_batch']
path_to_batch = seqsettings['path_to_batch']

deterministic = asettings['deterministic']
seed = asettings['deterministic_seed']
slowdown = asettings['slowdown']
use_threshold = asettings['use_threshold']
number_of_iterations = asettings['max_num_iterations']
num_changes_to_keep = asettings['num_changes_to_keep']
keep_threshold_percent = asettings['threshold_percent']

burn_in = bsettings['iterations_to_burnin']
burn_percent = bsettings['burn_threshold']
burn_num_to_keep = bsettings['burn_num_changes_to_keep']

import math
from IPython.utils.text import num_ini_spaces
import os

bi = burn_in

if deterministic:
  random.seed(seed)

AAs = "ARNDCEQGHILKMFPSTWYV"

original = original.strip()

def FindAllChar(str, char):
    lst = []
    for i in range(len(str)):
        if str[i] == char:
            lst.append(i)
    return lst

def CopyList(lst):
  ret = []
  for val in lst:
    ret.append(val)
  return ret

def ListIsEqual(l1, l2):
  if len(l1) != len(l2):
    return False
  for i in range(len(l1)):
    if l1[i] != l2[i]:
      return False
  return True

def Update(attempts, sequence, num_to_change = None, threshold = 0.0 ):
  if num_to_change is None or num_to_change == -1:
    num_to_change = len(attempts)
  attempts.sort(key = lambda x: x[1]['score'])
  attempts.reverse()
  updates = 0
  i = 0
  while i < len(attempts) and updates < num_to_change:
    index = attempts[i][0]
    if attempts[i][1]['sequence'].split()[index] != sequence[index] and attempts[i][1]['score'] > threshold:
      sequence[index] = attempts[i][1]['sequence'].split()[index]
      updates += 1
    i += 1
    if updates == num_to_change:
      return sequence, updates
  return sequence, updates

def Normalize(probabilities):
  factor = 1 / sum(probabilities)
  return [p*factor for p in probabilities]

def UpdateNoThresh(attempts, sequence, num_to_change=None, slow=False):
  updates = 0
  if not slow:
    for a in attempts:
      probabilities = [x[1]['score'] for x in a]
      chosen = random.choices(a, weights=probabilities, k=1)[0]
      old_char = sequence[chosen[0]]
      new_char = chosen[1]['sequence'].split()[chosen[0]]
      sequence[chosen[0]] = new_char
      if old_char != new_char:
        updates += 1
  else:
    flat = [a for attempt in attempts for a in attempt]
    probabilities = [x[1]['score'] for x in flat]
    normalized = Normalize(probabilities)
    choices = random.choices(flat, weights=normalized, k=num_to_change)
    for chosen in choices:
      old_char = sequence[chosen[0]]
      new_char = chosen[1]['sequence'].split()[chosen[0]]
      sequence[chosen[0]] = new_char
      if old_char != new_char:
        updates += 1
  return sequence, updates

def ScoreProbAverage(sequence, attempts):
  total = 0
  for a in attempts:
    char = sequence[a[0]]
    for unmask in a[1]:
      if unmask['token_str'] == char:
        total += unmask['score']
  return total / len(sequence)

def ScoreProbEntropy(sequence, attempts):
  total = 0
  for a in attempts:
    char = sequence[a[0]]
    for unmask in a[1]:
      if unmask['token_str'] == char:
        total += math.log(unmask['score'], 2) * unmask['score']
  return total * -1


def LogSetup(identifier, seq):
  logfile = open(f"logfile_{identifier}_{seq}.txt", "w")
  if use_custom_weights == "no":
    logfile.write("No custom weights used\n")
  if use_custom_weights != "From computer":
    logfile.write(f'Weights from:\t{path_to_file_in_colab}\n')
  else:
    logfile.write(f'Custom weights from personal machine\n')
  logfile.write(f"Original:\t{original}\n")
  logfile.write(f"Masked:\t{masked}\n\n")
  logfile.write(f"Order type:\t{order}\n")
  logfile.write(f"Deterministic:\t{deterministic}\n")
  if deterministic:
    logfile.write(f"Random seed:\t{seed}\n")
  # logfile.write(f"Randomize iterations:\t{str(randomize_every_iteration)}\n")
  logfile.write(f"Max iterations:\t{number_of_iterations}\n")
  if use_threshold:
    logfile.write(f"Max changes per iteration:\t{num_changes_to_keep}\n")
  else:
    logfile.write(f"Slowdown:\t{slowdown}\n")
  logfile.write(f"Threshold %:\t{keep_threshold_percent}\n\n")
  if burn_in > 0:
    logfile.write(f"Burn in iterations:\t{burn_in}\n")
    logfile.write(f"Burn in max changes:\t{burn_percent}\n")
    logfile.write(f"Burn in threshold %:\t{burn_num_to_keep}\n\n")
  return logfile

def LogSummary(logfile, iteration, original, new, data, datalog=5 ,score_avg=None, score_ent=None):
  # original = "".join(data[0][1]['sequence'])
  data.sort(key=lambda x:x[0])
  new = " ".join(new)
  # print(data)

  num_alpha = 0
  for c in original:
    if c.isalpha():
      num_alpha += 1
  num_str = ""
  for i in range(5, len("".join(original.split())) + 1, 5):
    if i == 5:
      num_str += f"        {i}"
      continue
    if len(str(i)) > len(str(i - 5)):
      num_str += " "
    num_str += "".join([" "] * (10 - len(str(i)))) + str(i)



  change_str = ""
  for i in range(len(original)):
    if original[i] != new[i]:
      change_str += "*"
    elif original[i].isalpha() and " ".join(masked)[i] == '_':
      change_str += '-'
    else:
      change_str += " "
  num_mask_str = ""
  num_masked = 0
  add_stack = []
  for i in range(len(change_str)):
    if change_str[i] in ['-', '*']:
      num_masked += 1
      if num_masked % 5 == 0:
        num_mask_str += str(num_masked)
        for _ in range(len(str(num_masked))):
          add_stack.append("")

    if len(add_stack) != 0:
      add_stack.pop()
    else:
      num_mask_str += " "


  orig_spl = original.split()
  new_spl = new.split()

  logfile.write(f"Iteration {iteration - 1}:\n")
  if score_avg is not None:
    logfile.write(f"Score (average): {score_avg}\n")
  if score_ent is not None:
    logfile.write(f"Score (entropy): {score_ent}\n")
  logfile.write(f"\t       {num_str}\n")
  logfile.write(f"\tStart: {original}\n")
  logfile.write(f"\t       {change_str}\n")
  logfile.write(f"\tEnd:   {new}\n")
  logfile.write(f"\t       {num_mask_str}\n")
  logfile.write(f"\t{change_str.count('*')} changes from previous start of iteration\n")
  logfile.write("\n")

  for mask in data:
    if orig_spl[mask[0]] != new_spl[mask[0]]:
      logfile.write("*")
    logfile.write(f"\tMask {mask[0] + 1}: {orig_spl[mask[0]]} -> {new_spl[mask[0]]}\n\t")
    for d in mask[1][:datalog]:
      logfile.write(f"\t{d['token_str']}: {d['score']:.7f}\t")
    logfile.write("\n")

if MLM_model:
  orig_batch = []
  mask_batch = []
  all_results = []

  scores_avg = [0.0]
  scores_ent = [0.0]


  if not process_batch:
    orig_batch.append(original)
    mask_batch.append(masked)
  else:
    with open(path_to_batch, "r") as fp:
      for line in fp.readlines():
        orig_batch.append(line.strip())
        mask_batch.append("")


  for i in range(len(orig_batch)):
    print("".join(["-"]*30))
    original = orig_batch[i]
    masked = mask_batch[i]
    ###
    assert all(c in AAs for c in original), f"Sequence must contain only letters corresponding to the 20 amino acids\n\ti.e. ARNDCEQGHILKMFPSTWYV\nProvided: {original}"
    if use_random_sequence:
      original = ""
      for v in range(random_sequence_length):
        original += AAs[random.randrange(0, len(AAs))]
      masked = "".join(["_"]*random_sequence_length)
    #Remove possible whitespace
    original = "".join(original.split())
    if match_original:
      masked = "_" * len(original)
    assert len(original) == len(masked), f"Original sequence and masked sequence must be the same length\n\tOriginal length: {len(original)}\n\tMasked length: {len(masked)}"
    ###

    import time
    t1 = time.time()

    if write_log:
      logfile = LogSetup(i, original[:3])



    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    unmasker = pipeline('fill-mask', model=model, tokenizer=tokenizer, device=device)

    ###preprocess###
    new_seq = ""
    col_seq = ""
    #Make a clean, standardized string using _ for masks
    for c in masked.replace('x', '_').replace(r'[MASK]', '_'):
      if not c.isspace():
        if spaces_in_output:
          col_seq += (c + " ")
        else:
          col_seq += c

    #Make masked string according to BERT standard
    new_seq = ' '.join(''.join(col_seq.split()))
    orig = ' '.join(''.join(original.split()))

    # print(orig)
    print("Generating from: ", original.replace(' ', ''), "\n", "         using: ", new_seq.replace(' ', ''))
    num_masked = "".join(new_seq).count("_")
    print(f"Number of masked residues: {num_masked}")
    modify = new_seq.split()

    locations = FindAllChar(modify, '_')
    if order == 'right-to-left':
      locations.reverse()
    elif order == 'random':
      random.shuffle(locations)

    orig_spl = orig.split()
    retain = orig.split()
    before = cp.deepcopy(orig_spl)


    iterations = []
    iterations.append("".join(orig_spl))
    pbar = tqdm(total=(number_of_iterations * len(locations)), position=0, leave=True)
    for _ in range(number_of_iterations):
      data = []
      iterations.append("".join(orig_spl))
      changes = []
      noThreshChanges = []
      if order == 'random':
        random.shuffle(locations)
      attempts = []
      for val in locations:
        orig_spl = cp.deepcopy(before)
        orig_spl[val] = '[MASK]'

        noThreshCont = []


        # if get_score:
        #   numlog = 30
        # elif num_to_log < 5:
        #   numlog = 5
        # else:
        #   numlog = num_to_log

        numlog = 30

        masked_seq = " ".join(orig_spl)
        # inputs = tokenizer(masked_seq, return_tensors='pt')
        # inputs.to(device)
        # unmask= unmasker(inputs, top_k=numlog)

        unmask= unmasker(masked_seq, top_k=numlog)

        attempts.append((val, unmask))

        data.append( (val, unmask) )
        for attempt in unmask:
          if attempt['token'] >= 5:
            orig_spl = before
            changes.append((val, attempt))
            if use_threshold:
              break
            else:
              noThreshCont.append((val, attempt))
        if not use_threshold:
          noThreshChanges.append(noThreshCont)
        pbar.update(1)
      retain = cp.deepcopy(before)

      score_avg = ScoreProbAverage(before, attempts)
      scores_avg.append(score_avg)
      if display_score_average:
        desc_avg = f" Current score (average): {score_avg}"
      else:
        desc_avg = ""


      score_ent = ScoreProbEntropy(before, attempts)
      scores_ent.append(score_ent)
      if display_score_entropy:
        desc_ent = f" Current score (entropy): {score_ent}"
      else:
        desc_ent = ""
      if bi > 0:
        bi -= 1
        if use_threshold:
          orig_spl, num_changed = Update(changes, before, threshold=burn_percent, num_to_change=burn_num_to_keep)
        else:
          orig_spl, num_changed = UpdateNoThresh(noThreshChanges, before, num_to_change=burn_num_to_keep, slow=slowdown)
      else:
        if use_threshold:
          orig_spl, num_changed = Update(changes, before, threshold=keep_threshold_percent, num_to_change=num_changes_to_keep)
        else:
          orig_spl, num_changed = UpdateNoThresh(noThreshChanges, before, num_to_change=num_changes_to_keep, slow=slowdown)
      # orig_spl, num_changed = Update(changes, before, threshold=keep_threshold_percent, num_to_change=num_changes_to_keep)
      description_str = f"{num_changed} changes from previous iteration." + desc_avg + desc_ent
      pbar.set_description(description_str)
      # pbar.set_description(f"{num_changed} changes from previous iteration. Current score{score_avg}")
      if write_log:
        LogSummary(logfile, len(iterations), "".join(changes[0][1]['sequence']), orig_spl, data, datalog=num_to_log, score_avg=score_avg, score_ent=score_ent)
      if ListIsEqual(retain, orig_spl) and use_threshold:
        print("\nConverged")
        all_results.append("".join(orig_spl))
        break

    pbar.close()

    iterations.append("".join(orig_spl))
    if write_log:
      print(f"Progress and results dumped to logfile_{i}.txt")
      logfile.close()

    if not ListIsEqual(retain, orig_spl):
      print(f"\nNo convergence in {number_of_iterations} iterations")


    if spaces_in_output:
      # print("Original: ", " ".join(retain))
      print("\nFinal Result:\n", " ".join(orig_spl))
    else:
      # print("Original: ", "".join(retain))
      print("\nFinal Result:\n", "".join(orig_spl))

    print("Time: ", time.time() - t1)


    def PrintColoredDifferences(l1, l2, color='red', fin = "", score_avg=None, score_ent=None):
      length = len(l1)
      num_diff = 0
      num_c = []
      assert len(l1) == len(l2), "DifferentSize error"
      for i in range(len(l1)):
        if l1[i] == l2[i]:
          print(f"{l2[i]}", end = fin)
        else:
          print(colored(f"{l2[i]}", 'red'), end = fin)
          num_diff += 1
      print(" | ", end="")
      if display_num_different:
        print(f" {num_diff:^{len(' Changes |') - 3}}", end= "|")
      if display_score_average:
        print(f" {score_avg:^{len('| Avg. Score |') - 3}.4f}", end="|")
      if display_score_entropy:
        print(f" {score_ent:^{len('| Avg. Score |') - 3}.4f}", end="|")

      print()
      return num_diff


    if display_iterations:
      print("Iterations:")
      num_c = []
      print(f"{'Sequence':^{1 + len(iterations[0])}}", end = "|")
      if display_num_different:
        print(" Changes |", end = "")
      if display_score_average:
        print(" Avg. Score |", end = "")
      if display_score_entropy:
        print(" Ent. Score |", end = "")
      print()
      for i in range(1, len(iterations)):
        if spaces_in_output:
          num_c.append(PrintColoredDifferences(iterations[i - 1], iterations[i], 'red', " ", score_avg=scores_avg[i - 1], score_ent=scores_ent[i - 1]))
        else:
          num_c.append(PrintColoredDifferences(iterations[i - 1], iterations[i], 'red', "", score_avg=scores_avg[i - 1], score_ent=scores_ent[i - 1]))
        if display_score_average:
          pass
        plt.plot(num_c[1:])
        plt.title("Number of Changes per Iteration")
        plt.xlabel('Iteration')
        plt.ylabel('Number of Changes')
      plt.show()


    if display_score_average:
      plt.plot(scores_avg)
      plt.title("Scores over each iteration (average)")
      plt.xlabel('Iteration')
      plt.ylabel('Score')
      plt.show()

    if display_score_entropy:
      plt.plot(scores_ent)
      plt.title("Scores over each iteration (entropy)")
      plt.xlabel('Iteration')
      plt.ylabel('Score')
      plt.show()


  # print("".join(["-"]*30))
  # print("Summary:")
  # for r in all_results:
  #   print(r) Okay, so I think I see the problem. Go to line 411 and change that "iterations[i - 1]" to "iterations[0]"